In [ ]:
%matplotlib ipympl

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pykonal

In [ ]:
solver = pykonal.EikonalSolver()
solver.mode = 'hybrid-cartesian'
solver.vgrid.min_coords     = 0, 0, 0
solver.vgrid.node_intervals = 1, 1, 1
solver.vgrid.npts           = 100, 100, 1
solver.vv                   = np.ones(solver.vgrid.npts)
solver.src_loc              = 50, 50, 0
solver.solve()
src_solver                  = solver.src_solver

uu0 = np.sqrt(np.sum(np.square(solver.pgrid[...] - solver.src_loc), axis=-1))
# np.any(np.isnan(solver.uu)), np.any(np.isinf(solver.uu))

In [ ]:
old = np.seterr(divide='ignore')

In [ ]:
np.seterr(**old)

In [ ]:
1 / np.array([1, 0])

In [ ]:
vmin, vmax = -0.2, 0.2
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, aspect=1)

xx = src_solver.pgrid[..., 0] * np.sin(src_solver.pgrid[..., 1]) * np.cos(src_solver.pgrid[..., 2]) + solver.src_loc[0]
yy = src_solver.pgrid[..., 0] * np.sin(src_solver.pgrid[..., 1]) * np.sin(src_solver.pgrid[..., 2]) + solver.src_loc[1]
zz = src_solver.pgrid[..., 0] * np.cos(src_solver.pgrid[..., 1]) + solver.src_loc[2]
uu0 = np.sqrt(np.square(xx-solver.src_loc[0]) + np.square(yy-solver.src_loc[1]) + np.square(zz-solver.src_loc[2]))
# qmesh = ax.pcolormesh(xx[:,0,:], yy[:,0,:], 
#     (src_solver.uu[:,0,:]-uu0[:,0,:])/uu0[:,0,:],
#     vmin=vmin,
#     vmax=vmax,
#     cmap=plt.get_cmap('jet_r')
# )

xx = solver.pgrid[..., 0]
yy = solver.pgrid[..., 1]
zz = solver.pgrid[..., 2]

uu0 = np.sqrt(np.square(xx-solver.src_loc[0]) + np.square(yy-solver.src_loc[1]) + np.square(zz-solver.src_loc[2]))
qmesh = ax.pcolormesh(xx[:,:,0], yy[:,:,0],
    ((solver.uu[:,:,0]-uu0[:,:,0])/uu0[:,:,0])*100,
    vmin=vmin,
    vmax=vmax,
    cmap=plt.get_cmap('jet_r')
)
ax.scatter(solver.src_loc[0], solver.src_loc[1], marker='x', color='k')

cbar = fig.colorbar(qmesh, ax=ax)

In [ ]:
vmin, vmax = 0, np.max(np.abs(solver.uu))

plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, aspect=1)

xx = solver.pgrid[..., 0]
yy = solver.pgrid[..., 1]
zz = solver.pgrid[..., 2]
qmesh = ax.pcolormesh(
    xx[:,:,0]-solver.pgrid.node_intervals[0]/2, 
    yy[:,:,0]-solver.pgrid.node_intervals[1]/2,
    solver.uu[:,:,0],
    vmin=vmin,
    vmax=vmax,
    cmap=plt.get_cmap('jet_r'),
)

xx = src_solver.pgrid[..., 0] * np.sin(src_solver.pgrid[..., 1]) * np.cos(src_solver.pgrid[..., 2]) + solver.src_loc[0]
yy = src_solver.pgrid[..., 0] * np.sin(src_solver.pgrid[..., 1]) * np.sin(src_solver.pgrid[..., 2]) + solver.src_loc[1]
zz = src_solver.pgrid[..., 0] * np.cos(src_solver.pgrid[..., 1]) + solver.src_loc[2]
qmesh = ax.scatter(xx[:,0,:], yy[:,0,:], 
    c=src_solver.uu[:,0,:],
    vmin=vmin,
    vmax=vmax,
    cmap=plt.get_cmap('jet_r'),
    edgecolor='k'
)

cbar = fig.colorbar(qmesh, ax=ax)

In [ ]:
np.argwhere(dd < 2)

In [ ]:
src_solver.pgrid[np.unravel_index(np.argmin(src_solver.uu), src_solver.uu.shape)]

In [ ]:
np.any(np.isnan(solver.uu)), np.any(np.isinf(solver.uu))

In [ ]:
solver.uu - np.sqrt(np.sum(np.square(solver.pgrid[...] - solver.src_loc), axis=-1))

In [ ]:
np.argwhere(solver.uu < 1.419)